In [1]:
# Pick directory --> load all data into study dict

# study --> A1, A2, ... --> A1S1, A1S2, ... A2S1, A2S2, ... --> A1S1C1, A1S1C2, ..., A1S2C1, ...

# sort cell data, match across waveforms

# batch run spikeAnalysisGUI (firing data) {batch_ratemaps.py}

# batch run spikeWidthGUI (spike width)

# batch run PowerSpectrum.vel --> save output LFP data and load into python

# batch run hfoGUI (LFP vizualization) 

# batch run PRISM/neurofunc (field properties) {batch_ratemaps.py}

# batch run odor/object analysis

In [2]:
""" ACTIVE JUPYTER NOTEBOOK TO BATCH RUN UNIT MATCHING ALGORITHM """

# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2


In [3]:
# Outside imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# Set necessary paths / make project path = ...../neuroscikit/
unit_matcher_path = os.getcwd()
prototype_path = os.path.abspath(os.path.join(unit_matcher_path, os.pardir))
project_path = os.path.abspath(os.path.join(prototype_path, os.pardir))
lab_path = os.path.abspath(os.path.join(project_path, os.pardir))
out_path = os.path.abspath(os.path.join(lab_path, os.pardir))
sys.path.append(project_path)
os.chdir(project_path)
print(project_path)

# Internal imports

# Read write modules
from x_io.rw.axona.batch_read import make_study
from _prototypes.unit_matcher.read_axona import read_sequential_sessions, temp_read_cut
from _prototypes.unit_matcher.write_axona import format_new_cut_file_name

# Unit matching modules
from _prototypes.unit_matcher.main import format_cut, run_unit_matcher, map_unit_matches_first_session, map_unit_matches_sequential_session
from _prototypes.unit_matcher.session import compare_sessions
from _prototypes.unit_matcher.waveform import time_index, derivative, derivative2, morphological_points

c:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit


In [4]:
""" If a setting is not used for your analysis (e.g. smoothing_factor), just pass in an arbitrary value or pass in 'None' """
STUDY_SETTINGS = {

    'ppm': 511,  # EDIT HERE

    'smoothing_factor': None, # EDIT HERE

    'useMatchedCut': False,  # EDIT HERE, set to False if you want to use runUnitMatcher, set to True after to load in matched.cut file
}


# Switch devices to True/False based on what is used in the acquisition (to be extended for more devices in future)
device_settings = {'axona_led_tracker': True, 'implant': True} 

# Make sure implant metadata is correct, change if not, AT THE MINIMUM leave implant_type: tetrode
implant_settings = {'implant_type': 'tetrode', 'implant_geometry': 'square', 'wire_length': 25, 'wire_length_units': 'um', 'implant_units': 'uV'}

# WE ASSUME DEVICE AND IMPLANT SETTINGS ARE CONSISTENCE ACROSS SESSIONS

# Set channel count + add device/implant settings
SESSION_SETTINGS = {
    'channel_count': 4, # EDIT HERE, default is 4, you can change to other number but code will check how many tetrode files are present and set that to channel copunt regardless
    'devices': device_settings, # EDIT HERE
    'implant': implant_settings, # EDIT HERE
}

STUDY_SETTINGS['session'] = SESSION_SETTINGS

settings_dict = STUDY_SETTINGS

In [5]:
data_dir = out_path + r'\20180530-ROUND-3300-1X2B3A' 

# To use in unit matching
settings_dict_unmatched = settings_dict
settings_dict_unmatched['useMatchedCut'] = False

In [6]:
# unmatched_study = run_unit_matcher([data_dir], settings_dict_unmatched)
# print('COMPLETED UNIT MATCHING')

In [7]:
study = make_study([data_dir], settings_dict_unmatched)
study.make_animals()

Animal ID set
Animal ID set
Animal ID set
Animal ID set
Session data added, spikes sorted by cell
Session data added, spikes sorted by cell
Session data added, spikes sorted by cell
Session data added, spikes sorted by cell


In [40]:
def aggregate_event_times(study):
    agg_events = []
    for animal in study.animals:
        for ses in animal.ensembles:
            for cell in animal.ensembles[ses].cells:
                agg_events.append(cell.event_times)
    return agg_events

In [41]:
agg_event_times = aggregate_event_times(study)

array([2.14583333e-02, 3.16270833e-01, 8.44020833e-01, ...,
       2.40162396e+02, 2.40180646e+02, 2.40461875e+02])